In [1]:
import pandas as pd
from pymongo import MongoClient
from src.geopoint_mongo_functions import geoQueryNear

In [2]:
starbuck_table = pd.read_csv('output/starbucks_table.csv', encoding ='latin-1')

In [3]:
starbuck_table.head(5)

,Unnamed: 0,name,location,geopoint
0,0,Milan Roastery,Piazza Cordusio 3 Milan 20123,"{'type': 'Point', 'coordinates': [9.1862, 45.4..."
1,1,Starbucks Porta Romana,"Via Lentasio, 1 angolo Corso di Porta Romana M...","{'type': 'Point', 'coordinates': [9.19167, 45...."
2,2,Starbucks Durini,"Via Durini, 28 Milan 20121","{'type': 'Point', 'coordinates': [9.19836, 45...."
3,3,Starbucks Turati,"Via Filippo Turati, 25 Milano 20121","{'type': 'Point', 'coordinates': [9.1949, 45.4..."
4,4,Starbucks Garibaldi,"Corso Garibaldi, 118 Milano 20121","{'type': 'Point', 'coordinates': [9.18336, 45...."


In [4]:
design_table = pd.read_csv('output/sitios_para_diversion_table.csv', encoding='latin-1')

In [5]:
design_table.head(5)

,Unnamed: 0,insegna,LONG_WGS84,LAT_WGS84,geopoint
0,0,PERSEFONE,9.175731,45.496658,"{'type': 'Point', 'coordinates': ['9.175731025..."
1,1,MAF CAFFE',9.187345,45.481580,"{'type': 'Point', 'coordinates': ['9.187344614..."
2,2,NEW ART CAFE',9.187446,45.472379,"{'type': 'Point', 'coordinates': ['9.187446034..."
3,3,BANA,9.247868,45.452775,"{'type': 'Point', 'coordinates': ['9.247867823..."
4,4,NOY CONCEPT RESTAURANT CAFE',9.162191,45.465265,"{'type': 'Point', 'coordinates': ['9.162190924..."


# Mongodb geospartial with near

In [6]:
dbName = "Italian_Companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [7]:
# llamo la funcíon <geoQueryNear>, que me permite crear un diccionario, 
# y así poder hacer una query a MongoDB en funcíon de un radius determinado por defecto o escojido
star = {'type': 'Point', 'coordinates': [9.1862, 45.46496]}
q = db.clean_civicos_milan.find(geoQueryNear(star, radius=1000))
near_offices = list(q)
len(near_offices)

4170

In [8]:
df = pd.DataFrame(near_offices)
df

,_id,NUMEROCOMPLETO,TIPO,DESCRITTIVO,LONG_WGS84,LAT_WGS84,geopoint
0,5ef7beea830187474738cd8a,1,Piazza,CORDUSIO,9.186174,45.464964,"{'type': 'Point', 'coordinates': [9.1861737492..."
1,5ef7beea830187474738cd72,1C01,Piazza,CORDUSIO,9.186174,45.465075,"{'type': 'Point', 'coordinates': [9.1861737492..."
2,5ef7beea830187474738c8f8,15,Via,OREFICI,9.186425,45.464914,"{'type': 'Point', 'coordinates': [9.1864251391..."
3,5ef7beea830187474738cd7f,1,Via,CORDUSIO,9.185877,45.464904,"{'type': 'Point', 'coordinates': [9.1858771255..."
4,5ef7beea830187474738c91a,13,Via,OREFICI,9.186513,45.464849,"{'type': 'Point', 'coordinates': [9.1865134867..."
...,...,...,...,...,...,...,...
4165,5ef7beeb830187474739980c,1,Largo,PAOLO GRASSI,9.182555,45.473564,"{'type': 'Point', 'coordinates': [9.1825548861..."
4166,5ef7beeb83018747473956ab,2N05,Corso,GENOVA,9.177252,45.458540,"{'type': 'Point', 'coordinates': [9.1772518327..."
4167,5ef7beea830187474738c3a8,20,Via,CERVA,9.198980,45.464438,"{'type': 'Point', 'coordinates': [9.1989799693..."
4168,5ef7beeb8301874747395002,6,Via,SAN VINCENZO,9.176809,45.458857,"{'type': 'Point', 'coordinates': [9.1768088923..."
